# Problem Statement

A clique in an undirected graph $𝐺=(𝑉,𝐸)$ is a subset of vertices $C \subset V$ such that every pair of vertices in $C$ is connected by an edge.
A maximal clique is a clique that cannot be extended by including any adjacent vertex , i.e., it’s not contained within any larger clique.

# Background
1. Let M(G) denote the set of all maximal cliques of G. It is known that there exist n-vertex graphs with $3^{n∕3}$ maximal cliques
2. Worst case of running pivot is $3^{n/3}$, but is faster in practice on sparse graphs

## Pruning Rules
1. Pivot in Bron–Kerbosch (Tomita variant)

- The pivot is used to minimize the number of recursive calls.
- A pivot vertex `u` is selected from `P ∪ X`, typically the one with the maximum number of neighbors in `P`
- The algorithm then explores only vertices in `P` that are *not* adjacent to `u`** (i.e., `P \ N(u)`).
- **Might not be possible in implement in our**


2. Remove vertices with degree less than current clique size, **can be easily implemented**
3. Connected Component Decomposition 
   - Decomposing `P` into connected subgraphs ensures each component is processed independently.
   - **Can be implemented on `ExpandTo`**.
4. If vertex v has a neighborhood that is a subset of vertex u’s neighborhood (i.e., $N(v)⊆N(u)$, then every clique that contains v also contains u.
Hence, v can never generate a new maximal clique that doesn’t already include u. **Can we used initially to shrink ExpandFrom**


# New Impl

**Intermediate Arrays**
- **ExpandFrom** → vertices used as starting points for expansion  
- **ExpandMid** → intermediate vertices paired with the current start vertex  
- **ExpandTo** → vertices used to expand from the current pair  
- **Visited** → vertices already used as starting points  


### **Algorithm Steps**

1. **Initialization**
   - `ExpandFrom ← all vertices`
   - `ExpandTo ← all vertices except the first`
   - `ExpandMid ← ∅`

2. **Termination Condition**
   - If `ExpandFrom` or `ExpandTo` is empty → **Stop**

3. **Expansion Phase**
   - Pick the first vertex `v` from `ExpandFrom`
   - Add `v` to the current clique

4. **Handle Intermediate Vertex**
   - If `ExpandMid ≠ ∅`:
     - Try adding the vertex in `ExpandMid` to the current clique  
     - If addition fails:
       - Remove that vertex from `ExpandMid`
       - Move the first vertex from `ExpandTo` into `ExpandMid`
       - **Recurse / Call Again**
     - Else (if addition succeeds):
       - Continue expanding

5. **Expand Using `ExpandTo`**
   - Try adding vertices from `ExpandTo` to the current clique one by one  
   - If addition fails for a vertex, skip to the next

6. **Clique Found**
   - If `|clique| > 0`:
     - Record the clique vertices  
     - `ExpandFrom ← clique vertices`
     - `ExpandMid ← first unvisited vertex in clique`
     - `ExpandTo ← remaining unvisited vertices`
     - Mark the current vertex as **Visited**
     - **Recurse / Call Again**

7. **No Clique Found (Mid not empty)**
   - Move the first vertex from `ExpandTo` → `ExpandMid`
   - **Recurse / Call Again**

8. **No Clique Found (Mid empty)**
   - Remove the first vertex from both `ExpandFrom` and `ExpandTo`
   - **Recurse / Call Again**


![img](pics/neworder.jpeg)

![img](pics/neworder2.jpeg)